# espy - error simulations in python


In [2]:
print("We simulate erorrs in python!")

We simulate erorrs in python!


# Project

 - Simulate bit transmission on  physical layer
 - Implement and understand FEC and BEC algorithms and CRC
 - Compare algorithms and combinations under different channel models

# Goals

- Understanding for physical layer and bit error handling
- Learn how to do discrete event simulation in python
- Have fun with python

# Noise simulation  
### Creating random test data

In [3]:
import numpy
from espy.algorithms.channel_noise_simulator import channel_noise_simulator

#set seed
numpy.random.seed(100)

cns = channel_noise_simulator()
print(cns.create_random_bits_string(20))

00111100000100001001


# Noise simulation  
### Simulate simple errors

In [4]:
import numpy
from espy.algorithms.channel_noise_simulator import channel_noise_simulator
#seed
numpy.random.seed(100)
cns = channel_noise_simulator()

data = cns.create_random_bits_string(20)
noisy_data = cns.randomise_bits_string(data,0.1)

print(data)
print(noisy_data)
# highlight differences
print(''.join(str(e) for e in cns.compare_and_highlight_differences(data,noisy_data)))

00111100000100001001
00111100000100000000
Differences found: 2
00000000000000001001


# Noise simulation  
### Simulate burst errors

In [11]:
import numpy
from espy.algorithms.channel_noise_simulator import channel_noise_simulator

numpy.random.seed(31)
cns = channel_noise_simulator()

data = cns.create_random_bits_string(100)
noisy_data = cns.randomise_bits_burst_string(data,0.01,0.9)

print(data)
print(noisy_data)

# highlight differences
print(''.join(str(e) for e in cns.compare_and_highlight_differences(data,noisy_data)))

0100010001000000000111101010001101100110110101100011111100000101010111100101001001001101101101000100
0111101110111111111000010101110010011001001010000011111100000101010111100101001001001101101101000000
Differences found: 46
0011111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000100


In [24]:
import numpy
from espy.algorithms.channel_noise_simulator import channel_noise_simulator
numpy.random.seed(42)

cns = channel_noise_simulator()
example_data = ""
sample_size = 40000
for i in range(sample_size):
    example_data += "0"

EDF = 2 #Error density multiplier higher value means smaler but more errors
x1 = 0
x2 = 0
x3 = 0
count = 100

enter_error_rate=0.01;#chance to start bursting
leave_error_rate=0.05;#chance to stop bursting

for i in range(count):
    error_burst1=cns.randomise_bits_burst_string(example_data, enter_error_rate      ,1-(leave_error_rate))
    error_burst2=cns.randomise_bits_burst_string(example_data, enter_error_rate*EDF  ,1-(leave_error_rate*EDF))
    
    x1+=error_burst1.count('1')
    x2+=error_burst2.count('1')

print(x1/count/sample_size)
print(x2/count/sample_size)

0.15966075000000002
0.1530115


# Cyclic Redundancy Check

 - Interpret the message as a polynomial $M(x)$
 - Choose an irreducible polylomial $C(x)$ with the degree $n$
 - Compute modulus $R(x) = M(x)\cdot x^n \mod C(x)$
 - Send the message and $R$
 - Check if the received messages modulus is equal to $R$

# Cyclic Redundancy Check

In [9]:
from espy.crc import CyclicRedundancyCheck

crc3 = CyclicRedundancyCheck("1101")
cns = channel_noise_simulator()
msg = cns.create_random_bits_string(20)

msg_with_crc = crc3.crc_encode(msg)
msg_with_crc_noisy = cns.randomise_bits_string(msg_with_crc, 0.0)

print(msg)
print(msg_with_crc)
print(msg_with_crc_noisy)
print(''.join(str(e) for e in cns.compare_and_highlight_differences(msg_with_crc, msg_with_crc_noisy)))

10001101101010011010
10001101101010011010101
10001101101010011010101
Differences found: 0
00000000000000000000000


In [10]:
print(crc3.crc_check(msg_with_crc_noisy))

True


# Block Codes

In [11]:
from espy.block_code import *
import numpy as np

generator = np.array([[1, 0, 0, 0, 1, 1, 0],
                      [0, 1, 0, 0, 1, 0, 0],
                      [0, 0, 1, 0, 0, 1, 1],
                      [0, 0, 0, 1, 0, 0, 0]])

# init systematic block code with generator matrix
systematic_code = SystematicCode(generator)

# check encode
message = np.array([1, 0, 0, 1])
codeword = systematic_code.encode(message)
print(codeword == [1, 0, 0, 1, 1, 1, 1])

# check decode
decoded_message = systematic_code.decode(codeword)
print(decoded_message == message)



[ True  True  True  True  True  True False]
[ True False False False]


In [12]:
from espy.block_code import *
import numpy as np

# init systematic block code with generator matrix
systematic_code = SystematicCode(generator)
# check code word table
codeword_table = systematic_code.codeword_table()
print(codeword_table)
# check validate
print(systematic_code.validate(codeword))
print(not systematic_code.validate([1, 1, 1, 1, 0, 0, 0]))

[[0 0 0 0 0 0 0]
 [1 0 0 0 1 1 0]
 [0 1 0 0 1 0 0]
 [1 1 0 0 0 1 0]
 [0 0 1 0 0 1 1]
 [1 0 1 0 1 0 1]
 [0 1 1 0 1 1 1]
 [1 1 1 0 0 0 1]
 [0 0 0 1 0 0 0]
 [1 0 0 1 1 1 0]
 [0 1 0 1 1 0 0]
 [1 1 0 1 0 1 0]
 [0 0 1 1 0 1 1]
 [1 0 1 1 1 0 1]
 [0 1 1 1 1 1 1]
 [1 1 1 1 0 0 1]]
True
True


# ARQ Protocol Simulation

In [13]:
import simpy, logging
from espy.simulation.client import Client
from espy.simulation.router import Router

NUM_CLIENTS = 3
SIM_TOTAL_DURATION = 1000
ROUTER_DROP_PROB = 0.3
SPEED = 0.2
LOG_LEVEL = logging.INFO

def init_clients():
    clients = []
    for i in range(NUM_CLIENTS):
        clients.append(Client(env, i, router, NUM_CLIENTS))
    return clients


logging.basicConfig(format='[%(levelname)-7s] %(name)8s: %(message)s', level=LOG_LEVEL)

# init simpy and network
env = simpy.Environment()
router = Router(env, ROUTER_DROP_PROB, SPEED)
clients = init_clients()
router.store_clients(clients)

In [14]:
logging.info("Simulation starting")
env.run(until=SIM_TOTAL_DURATION)
logging.info("Simulation finished")

[INFO   ]     root: Simulation starting
[INFO   ] Client 0: Send message to 2: Hello World!
[WARNING]   Router: Dropped packet from 0 to 2: Hello World!
[INFO   ] Client 1: Send message to 0: Hello World!
[WARNING]   Router: Dropped packet from 1 to 0: Hello World!
[INFO   ] Client 2: Send message to 1: Hello World!
[INFO   ] Client 1: Received message from 2: Hello World!
[INFO   ] Client 1: Send message to 2: ACK
[WARNING]   Router: Dropped packet from 1 to 2: ACK
[WARNING] Client 0: Did not receive ACK. Resending...
[INFO   ] Client 0: Send message to 2: Hello World!
[WARNING]   Router: Dropped packet from 0 to 2: Hello World!
[WARNING] Client 1: Did not receive ACK. Resending...
[INFO   ] Client 1: Send message to 0: Hello World!
[INFO   ] Client 0: Received message from 1: Hello World!
[INFO   ] Client 0: Send message to 1: ACK
[INFO   ] Client 1: Received message from 0: ACK
[WARNING] Client 2: Did not receive ACK. Resending...
[INFO   ] Client 2: Send message to 1: Hello World!


[INFO   ]     root: Simulation starting  
[INFO   ] Client 0: Send message to 1: Hello World!  
[INFO   ] Client 1: Received message from 0: Hello World!  
[INFO   ] Client 1: Send message to 0: ACK    
[INFO   ] Client 0: Received message from 1: ACK  
[INFO   ] Client 1: Send message to 2: Hello World!    
[WARNING]   Router: Dropped packet from 1 to 2: Hello World!  
[INFO   ] Client 2: Send message to 1: Hello World!  
[WARNING]   Router: Dropped packet from 2 to 1: Hello World!  
[WARNING] Client 1: Did not receive ACK. Resending...  
[INFO   ] Client 1: Send message to 2: Hello World!  
[INFO   ] Client 2: Received message from 1: Hello World!  
[INFO   ] Client 2: Send message to 1: ACK  
[INFO   ] Client 1: Received message from 2: ACK  
[WARNING] Client 2: Did not receive ACK. Resending...  
[INFO   ] Client 2: Send message to 1: Hello World!  
[WARNING]   Router: Dropped packet from 2 to 1: Hello World!  
[WARNING] Client 2: Did not receive ACK. Resending...  
[INFO   ] Client 2: Send message to 1: Hello World!  
[INFO   ] Client 1: Received message from 2: Hello World!  
[INFO   ] Client 1: Send message to 2: ACK  
[INFO   ] Client 2: Received message from 1: ACK  
[INFO   ]     root: Simulation finished  